<a href="https://colab.research.google.com/github/Corprey/Bachelorarbeit/blob/main/RL_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reinforcement Learning Agent and Environment

## Download and adjust Dataset (only for Colab)

In [ ]:
# Path to your kaggle.json
kaggleFile = "/content/kaggle.json"

In [ ]:
# Move kaggle.json to "/root/.kaggle" and set permissions
!pip install kaggle
!mkdir /root/.kaggle
!cp $kaggleFile /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Download dataset via API
!cd /content
!kaggle datasets download -d philippvanek/gtsdb-yolov5-format

100% 4.77G/4.79G [01:10<00:00, 78.5MB/s]
100% 4.79G/4.79G [01:10<00:00, 72.5MB/s]


In [ ]:
# Unzip dataset at specified location
!mkdir /content/datasets
!mv /content/gtsdb-yolov5-format.zip /content/datasets
!unzip /content/datasets/gtsdb-yolov5-format.zip -d /content/datasets

Archive:  /content/datasets/gtsdb-yolov5-format.zip
  inflating: /content/datasets/GTSDB/gt.txt  
  inflating: /content/datasets/GTSDB/images/00000.png  
  inflating: /content/datasets/GTSDB/images/00001.png  
  inflating: /content/datasets/GTSDB/images/00002.png  
  inflating: /content/datasets/GTSDB/images/00003.png  
  inflating: /content/datasets/GTSDB/images/00004.png  
  inflating: /content/datasets/GTSDB/images/00005.png  
  inflating: /content/datasets/GTSDB/images/00006.png  
  inflating: /content/datasets/GTSDB/images/00007.png  
  inflating: /content/datasets/GTSDB/images/00008.png  
  inflating: /content/datasets/GTSDB/images/00009.png  
  inflating: /content/datasets/GTSDB/images/00010.png  
  inflating: /content/datasets/GTSDB/images/00011.png  
  inflating: /content/datasets/GTSDB/images/00012.png  
  inflating: /content/datasets/GTSDB/images/00013.png  
  inflating: /content/datasets/GTSDB/images/00014.png  
  inflating: /content/datasets/GTSDB/images/00015.png  
  infl

## Install and define dependencies

In [ ]:
!pip install gymnasium
import gymnasium as gym
import sys
sys.modules["gym"] = gym

!pip install git+https://github.com/Stable-Baselines-Team/stable-baselines3-contrib@feat/gymnasium-support
!pip install stable-baselines3[extra]
!pip install yolov5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 925.5/925.5 kB 46.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/Stable-Baselines-Team/stable-baselines3-contrib (to revision feat/gymnasium-support) to /tmp/pip-req-build-e_1__1bl
  Running command git clone --filter=blob:none --quiet https://github.com/Stable-Baselines-Team/stable-baselines3-contrib /tmp/pip-req-build-e_1__1bl
  Running command git checkout -b feat/gymnasium-support --track origin/feat/gymnasium-support
  Switched to a new branch 'feat/gymnasium-support'
  Branch 'feat/gymnasium-support' set up to track remote branch 'feat/gymnasium-support' from 'origin'.
  Resolved https://github.com/Stable-Baselines-Team/stable-baselines3-contrib to commit d93ca7f1a734436cee59d642fa7d3eb8b4ed0253
  Installing build dependencies ... done
  Get

In [ ]:
import gymnasium as gym
from gymnasium import Env
from gymnasium.spaces import Box
import numpy as np
import random
import os
from PIL import Image
from PIL import ImageEnhance
import torch
import sys
sys.modules["gym"] = gym
from stable_baselines3 import SAC
import pandas as pd
import gc

In [ ]:
# clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5  # clone repo

Cloning into 'yolov5'...
remote: Enumerating objects: 15996, done.
remote: Counting objects: 100% (165/165), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 15996 (delta 99), reused 112 (delta 69), pack-reused 15831
Receiving objects: 100% (15996/15996), 14.65 MiB | 21.99 MiB/s, done.
Resolving deltas: 100% (10968/10968), done.


In [ ]:
# Path to training images
train_image_path = "/content/datasets/GTSDB/images/distorted_color"
original_image_path = "/content/datasets/GTSDB/images"
label_path = "/content/datasets/GTSDB/labels"

## Build Environment

Action: The Agent is able to use continous values to change each image parameter (Sharpness, Brightness, Contrast, Color)

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/YOLOv5_best_1000ep.pt')

/usr/local/lib/python3.10/dist-packages/torch/hub.py:286: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
WARNING ⚠️ user config directory is not writeable, defaulting to '/tmp/Ultralytics'.
YOLOv5 🚀 2023-6-12 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)



[Errno 2] No such file or directory: '/usr/local/lib/python3.10/dist-packages/pyparsing-3.0.9.dist-info/METADATA'


Fusing layers... 
Model summary: 213 layers, 1817344 parameters, 0 gradients, 4.3 GFLOPs
Adding AutoShape... 


In [ ]:
# Images
img = Image.open("../../GTSDB/images/00000.png")

# Inference
temp = np.asarray(img)
results = model(temp)

# Results
temp2 = results.pandas().xyxy[0]  # Pandas DataFrame
print(temp2)

In [ ]:
class DistortionEnv(Env):

    image_width = 320
    image_height = 320

    
    # Resizes the image to specified dimensions and converts to 3 color channels
    @staticmethod
    def _load_and_convert_image(image_path):
        image = Image.open(image_path)
        image = image.resize((DistortionEnv.image_width, DistortionEnv.image_height))
        if image.mode == "RGBA":
            image = image.convert("RGB")
        return np.asarray(image)

    @staticmethod
    def _calculate_mse(image1, image2):
   
        # Resize the images if necessary (to ensure they have the same dimensions)
        image1 = image1.resize((DistortionEnv.image_width, DistortionEnv.image_height))
        image2 = image2.resize((DistortionEnv.image_width, DistortionEnv.image_height))
    
        # Convert the images to grayscale
        image1 = image1.convert("L")
        image2 = image2.convert("L")
    
        # Convert the images to numpy arrays
        arr1 = np.array(image1)
        arr2 = np.array(image2)
    
        # Calculate the MSE
        mse = np.mean((arr1 - arr2) ** 2)
        return mse

    # Convert YOLOv5 format labeled bounding boxes to coordinates
    @staticmethod
    def _convert_yolo_to_coordinates(yolo_box, image_width, image_height):
        net_class, center_x, center_y, width, height = yolo_box
        
        xmin = (center_x - width/2) * image_width
        ymin = (center_y - height/2) * image_height
        xmax = (center_x + width/2) * image_width
        ymax = (center_y + height/2) * image_height
        return net_class, xmin, ymin, xmax, ymax

    def _get_gt_coords_from_yolo_file(self, file_path):
        yolo_boxes = []
        if os.path.exists(file_path):
            with open(file_path, 'r') as file:
                for line in file:
                    line = line.strip()  
                    net_class, center_x, center_y, width, height = line.split()     
                    
                    center_x = float(center_x)
                    center_y = float(center_y)
                    width = float(width)
                    height = float(height)
                    
                    entry = (net_class, center_x, center_y, width, height)
                    yolo_boxes.append(entry)

        # Convert Yolo bounding boxes to coordinates
        gt_coords = []
        for yolo_box in yolo_boxes:
            box_coords = self._convert_yolo_to_coordinates(yolo_box, DistortionEnv.image_width, DistortionEnv.image_height)
            gt_coords.append(box_coords)
        return gt_coords
        
    @staticmethod
    def _calculate_iou(box1, box2):
        class1, xmin1, ymin1, xmax1, ymax1 = box1
        class2, xmin2, ymin2, xmax2, ymax2 = box2

        if (str(class1) != str(class2)):
            return 0
        
        # Calculate intersection area
        xmin_inter = max(xmin1, xmin2)
        ymin_inter = max(ymin1, ymin2)
        xmax_inter = min(xmax1, xmax2)
        ymax_inter = min(ymax1, ymax2)
    
        width_inter = max(0, xmax_inter - xmin_inter)
        height_inter = max(0, ymax_inter - ymin_inter)
        area_inter = width_inter * height_inter
    
        # Calculate union area
        area_box1 = (xmax1 - xmin1) * (ymax1 - ymin1)
        area_box2 = (xmax2 - xmin2) * (ymax2 - ymin2)
        area_union = area_box1 + area_box2 - area_inter
    
        # Calculate IoU
        iou = area_inter / area_union if area_union > 0 else 0
        return iou


    def _check_ious(self, image_width, image_height, gt_coords, predictions):
        # Iterate over prediction rows, calculate IoU with each ground truth box, keep highest IoU
        iou_values = []
        for index, row in predictions.iterrows():
            predicted_box = (row['class'], row['xmin'], row['ymin'], row['xmax'], row['ymax'])
        
            # Calculate IoU with each ground truth box
            iou_per_gt = []
            for gt_coord in gt_coords:
                iou = self._calculate_iou(gt_coord, predicted_box)
                iou_per_gt.append(iou)
        
            # Store the maximum IoU value for the predicted box
            if(len(iou_per_gt) != 0):
                max_iou = max(iou_per_gt)
                iou_values.append(max_iou)

        if (len(iou_values) != 0):
            mean_iou = sum(iou_values) / len(iou_values)
        else:
            if(len(gt_coords) == 0 and len(predictions) == 0):
                mean_iou = 1.0
            else:
                mean_iou = 0.0
        
        return mean_iou

    
    def __init__(self):
        # Load YOLOv5 model
        self.model = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/YOLOv5_best_1000ep.pt')
        
        # Define the action space bounds for sharpness, contrast, brightness and color
        sharpness_bounds = (0.5, 1.5)
        contrast_bounds = (0.5, 1.5)
        brightness_bounds = (0.5, 1.5)
        color_bounds = (0.5, 1.5)
        num_actions = 4

        # Use when multiple actions are possible
        self.action_space = Box(low = np.array([sharpness_bounds[0], contrast_bounds[0], brightness_bounds[0], color_bounds[0]]),
                               high = np.array([sharpness_bounds[1], contrast_bounds[1], brightness_bounds[1], color_bounds[1]]),
                               shape = (num_actions,), 
                               dtype = 'float32')

        # TODO: Simplified to only learn adjusting the brightness. Has to be changed later
        #self.action_space = Box(low = 0.6,
        #                       high = 1.4,
        #                       shape = (1,), 
        #                       dtype = 'float32')

        # Define the observation space for an image
        image_shape = (DistortionEnv.image_width, DistortionEnv.image_height, 3)  # (height, width, channels)
        image_dtype = np.uint8 

        # Load training images
        self.train_images = os.listdir(train_image_path)

        self.observation_space = Box(low = 0, high = 255, shape = image_shape, dtype = np.uint8)

    def reset(self, seed=None):
        # TODO: Set duration? e.g. 10 consecutive actions possible, maybe should start with only 1
        self.remaining_actions = 1

        # Choose random image for this episode
        self.image_name = random.choice(self.train_images)
        
        # Load input image
        input_image_path = os.path.join(train_image_path, self.image_name)
        self.image_input = self._load_and_convert_image(input_image_path)
        
        # Load original image
        #orig_image_path = os.path.join(original_image_path, self.image_name)
        #self.image_original = self._load_and_convert_image(orig_image_path)
        
        # Load ground truth (label and bounding box file)
        label_filename = self.image_name.split('.')[0] + ".txt"
        self.gt_coords = self._get_gt_coords_from_yolo_file(os.path.join(label_path, label_filename))

        self.state = self.image_input.copy()
        return self.state, {}
        
    def step(self, action):
        self.remaining_actions -= 1
        
        # change image parameters according to action 
        distortion_factors = action
        enhancer = ImageEnhance.Sharpness(Image.fromarray(self.state))
        enhanced_image = enhancer.enhance(distortion_factors[0])
        enhancer = ImageEnhance.Contrast(enhanced_image)
        enhanced_image = enhancer.enhance(distortion_factors[1])
        enhancer = ImageEnhance.Brightness(enhanced_image)
        enhanced_image = enhancer.enhance(distortion_factors[2])
        enhancer = ImageEnhance.Color(enhanced_image)
        enhanced_image = enhancer.enhance(distortion_factors[3])
        self.state = np.asarray(enhanced_image)

        
        # use detection model on improved image
        results = self.model(self.state)
        predictions = results.pandas().xyxy[0]
        mean_iou_improved = self._check_ious(DistortionEnv.image_width, DistortionEnv.image_height, self.gt_coords, predictions)
        
        # use detection model on input image
        results = self.model(self.image_input)
        predictions = results.pandas().xyxy[0]
        mean_iou_input = self._check_ious(DistortionEnv.image_width, DistortionEnv.image_height, self.gt_coords, predictions)

        reward = 1 if (mean_iou_improved > mean_iou_input) else -1

        observation = self.state

        if (self.remaining_actions <= 0):
          done = True
        elif (mean_iou_improved > mean_iou_input):
          done = True
        else:
          done = False

        info = {} # Placeholder
        
        return observation, reward, done, False, info 
        
    def render(self):
        pass

In [ ]:
env = DistortionEnv()

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-6-19 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 213 layers, 1817344 parameters, 0 gradients, 4.3 GFLOPs
Adding AutoShape... 


In [ ]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    while not done:
        action = env.action_space.sample()
        n_state, reward, done, _, info = env.step(action)
        score += reward
    print("Episode:{} Score:{}".format(episode, score))

In [ ]:
model = SAC("MlpPolicy", env, verbose=1, buffer_size=10, batch_size=64)
model.learn(total_timesteps=5000, log_interval=5)
model.save("sac_yolov5_det")

In [ ]:
#del model
gc.collect()
torch.cuda.empty_cache()

In [ ]:
!nvidia-smi

Mon Jun 19 11:37:15 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    30W /  70W |   5819MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cp -r /content/sac_yolov5_det.zip /content/drive/MyDrive